In [ ]:
# Importing libraries needed for the project
import pandas as pd
import numpy as np
#from scipy.stats import pearsonr
#from scipy.stats import linregress
#import matplotlib.pyplot as plt
#import plotly.express as px
#from math import ceil #The math.ceil() method rounds a number UP to the nearest integer, if necessary, and returns the result.
#from urllib.request import urlopen
import json
#from matplotlib.ticker import MaxNLocator
#import seaborn as sns
#from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning

In [ ]:
focusareas = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

In [ ]:
# Load the Excel file into a DataFrame
df = pd.read_excel("./Per_Question_Time_Report_4142128-2.xls")

# Save the DataFrame as a CSV file
df.to_csv("./Per_Question_Time_Report_4142128-2.csv", index=False)

# Display the first few rows (optional, just to check the data)
df.head()
